# Aequitas

In [85]:
from aequitas import Audit

import aequitas.plot as ap
import pandas as pd

In [86]:
columns_to_select = [
    "sex", "age_cat", "race", "juv_fel_count", "decile_score", "juv_misd_count", "juv_other_count", 
    "c_charge_degree", "is_recid", "r_charge_degree", "is_violent_recid", 
    "vr_charge_degree", "score_text", "v_score_text", "two_year_recid", "days_b_screening_arrest",
    "score_factor", "predicted_score"
]

df = pd.read_csv("compas_with_predictions.csv")[columns_to_select]



In [87]:
# drop rows with missing values
df = df.dropna()

# drop row with race = Asian
df = df[df['race'] != 'Asian']

In [88]:
df = df.drop(columns=['days_b_screening_arrest'])

In [89]:
# convert columns to categorical
for column in df.columns:
    df[column] = df[column].astype('object')
    
# convert target columns to numerical

df['score_factor'] = df['score_factor'].astype('int')

In [90]:
# select a row where race = Caucasian, sex = Male, age_cat = 25-45, recidivism = no, c_charge_degree = F, decile_score = 5, juv_fel_count = 0, juv_misd_count = 0, juv_other_count = 0, r_charge_degree = F, vr_charge_degree = F, score_text = Low, v_score_text = Low, two_year_recid = 0
# this row will be used as reference group
reference = df.loc[(df['race'] == 'Caucasian') & (df['sex'] == 'Male') & (df['age_cat'] == '25 - 45') & (df['score_text'] == 'Low')]
                       
                       
reference = reference.iloc[0]
reference.drop(['predicted_score', 'score_factor'], inplace=True)

reference = reference.to_dict()
reference

{'sex': 'Male',
 'age_cat': '25 - 45',
 'race': 'Caucasian',
 'juv_fel_count': 0,
 'decile_score': 3,
 'juv_misd_count': 0,
 'juv_other_count': 0,
 'c_charge_degree': 'M',
 'is_recid': 1,
 'r_charge_degree': '(M2)',
 'is_violent_recid': 1,
 'vr_charge_degree': '(M1)',
 'score_text': 'Low',
 'v_score_text': 'Low',
 'two_year_recid': 1}

In [91]:

audit = Audit(df, 
              label_column="predicted_score", 
              score_column="score_factor", 
              sensitive_attribute_column=["sex", "age_cat", "race", "is_recid", "is_violent_recid", "c_charge_degree", "decile_score", "juv_fel_count", "juv_misd_count", "juv_other_count", "r_charge_degree", "vr_charge_degree", "score_text", "v_score_text", "two_year_recid"],
              reference_groups=reference)
audit.audit()

In [92]:
audit.confusion_matrix

pp   pn  fp  fn   tn   tp  group_label_pos  \
attribute_name attribute_value                                                
sex            Female            49   33  10   6   27   39               45   
               Male             401  206  62  69  137  339              408   
age_cat        25 - 45          274  150  51  36  114  223              259   
               Greater than 45   30   52  15   6   46   15               21   
               Less than 25     146   37   6  33    4  140              173   
...                             ...  ...  ..  ..  ...  ...              ...   
v_score_text   High             140    1  13   1    0  127              128   
               Low              120  206  31  52  154   89              141   
               Medium           190   32  28  22   10  162              184   
two_year_recid 0                 21   19  12   4   15    9               13   
               1                429  220  60  71  149  369              440   

                                group_label_neg  group_size  total_entities  
attribute_name attribute_value                                               
sex            Female                        37          82             689  
               Male                         199         607             689  
age_cat        25 - 45                      165         424             689  
               Greater than 45               61          82             689  
               Less than 25                  10         183             689  
...                                         ...         ...             ...  
v_score_text   High                          13         141             689  
               Low                          185         326             689  
               Medium                        38         222             689  
two_year_recid 0                             27          40             689  
               1                            209         649             689  

[74 rows x 10 columns]

In [93]:
audit.metrics.round(2)

accuracy   tpr   tnr   for   fdr   fpr   fnr  \
attribute_name attribute_value                                                 
sex            Female               0.80  0.87  0.73  0.18  0.20  0.27  0.13   
               Male                 0.78  0.83  0.69  0.33  0.15  0.31  0.17   
age_cat        25 - 45              0.79  0.86  0.69  0.24  0.19  0.31  0.14   
               Greater than 45      0.74  0.71  0.75  0.12  0.50  0.25  0.29   
               Less than 25         0.79  0.81  0.40  0.89  0.04  0.60  0.19   
...                                  ...   ...   ...   ...   ...   ...   ...   
v_score_text   High                 0.90  0.99  0.00  1.00  0.09  1.00  0.01   
               Low                  0.75  0.63  0.83  0.25  0.26  0.17  0.37   
               Medium               0.77  0.88  0.26  0.69  0.15  0.74  0.12   
two_year_recid 0                    0.60  0.69  0.56  0.21  0.57  0.44  0.31   
               1                    0.80  0.84  0.71  0.32  0.14  0.29  0.16   

                                 npv  precision   ppr  pprev  prev  
attribute_name attribute_value                                      
sex            Female           0.82       0.80  0.11   0.60  0.55  
               Male             0.67       0.85  0.89   0.66  0.67  
age_cat        25 - 45          0.76       0.81  0.61   0.65  0.61  
               Greater than 45  0.88       0.50  0.07   0.37  0.26  
               Less than 25     0.11       0.96  0.32   0.80  0.95  
...                              ...        ...   ...    ...   ...  
v_score_text   High             0.00       0.91  0.31   0.99  0.91  
               Low              0.75       0.74  0.27   0.37  0.43  
               Medium           0.31       0.85  0.42   0.86  0.83  
two_year_recid 0                0.79       0.43  0.05   0.52  0.32  
               1                0.68       0.86  0.95   0.66  0.68  

[74 rows x 12 columns]

In [94]:
metrics = ['fpr','fdr']
disparity_tolerance = 1.25

In [95]:
audit.disparities.style

In [96]:
audit.disparity_plot(metrics=metrics, attribute='race', fairness_threshold=disparity_tolerance)

alt.LayerChart(...)

In [97]:
from aequitas.group import Group

group = Group()
xtab = group.get_crosstabs(df, score_col='score_factor', label_col='predicted_score', attr_cols = ["sex", "age_cat", "race", "is_recid", "is_violent_recid", "c_charge_degree", "decile_score", "juv_fel_count", "juv_misd_count", "juv_other_count", "r_charge_degree", "vr_charge_degree", "score_text", "v_score_text", "two_year_recid"])

xtab = xtab[0]
xtab

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev
0,0,binary 0/1,450,sex,Female,0.804878,0.866667,0.729730,0.181818,0.204082,...,0.597561,10,6,27,39,45,37,82,689,0.548780
1,0,binary 0/1,450,sex,Male,0.784185,0.830882,0.688442,0.334951,0.154613,...,0.660626,62,69,137,339,408,199,607,689,0.672158
2,0,binary 0/1,450,age_cat,25 - 45,0.794811,0.861004,0.690909,0.240000,0.186131,...,0.646226,51,36,114,223,259,165,424,689,0.610849
3,0,binary 0/1,450,age_cat,Greater than 45,0.743902,0.714286,0.754098,0.115385,0.500000,...,0.365854,15,6,46,15,21,61,82,689,0.256098
4,0,binary 0/1,450,age_cat,Less than 25,0.786885,0.809249,0.400000,0.891892,0.041096,...,0.797814,6,33,4,140,173,10,183,689,0.945355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,0,binary 0/1,450,v_score_text,High,0.900709,0.992188,0.000000,1.000000,0.092857,...,0.992908,13,1,0,127,128,13,141,689,0.907801
70,0,binary 0/1,450,v_score_text,Low,0.745399,0.631206,0.832432,0.252427,0.258333,...,0.368098,31,52,154,89,141,185,326,689,0.432515
71,0,binary 0/1,450,v_score_text,Medium,0.774775,0.880435,0.263158,0.687500,0.147368,...,0.855856,28,22,10,162,184,38,222,689,0.828829
72,0,binary 0/1,450,two_year_recid,0,0.600000,0.692308,0.555556,0.210526,0.571429,...,0.525000,12,4,15,9,13,27,40,689,0.325000


In [98]:
from aequitas.bias import Bias

bias = Bias()
hbdf = bias.get_disparity_predefined_groups(df=xtab, original_df=df, ref_groups_dict=reference, alpha=0.05, mask_significance=True)

In [99]:
hbdf[['attribute_name', 'attribute_value'] +
     bias.list_disparities(hbdf) + bias.list_significance(hbdf)]

,attribute_name,attribute_value,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity
0,sex,Female,0.122195,0.904537,0.941485,1.319947,0.542819,0.867480,0.788406,1.043068,1.059972,1.230259
1,sex,Male,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,age_cat,25 - 45,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,age_cat,Greater than 45,0.109489,0.566139,0.614350,2.686275,0.480769,0.795564,2.055556,0.829596,1.091458,1.163968
4,age_cat,Less than 25,0.532847,1.234574,1.178205,0.220790,3.716216,1.941176,1.372351,0.939890,0.578947,0.142248
...,...,...,...,...,...,...,...,...,...,...,...,...
69,v_score_text,High,1.166667,2.697400,1.223114,0.359447,3.961538,5.967742,0.021184,1.571893,0.000000,0.000000
70,v_score_text,Low,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
71,v_score_text,Medium,1.583333,2.325075,1.149616,0.570458,2.723558,4.397284,0.324206,1.394846,0.316131,0.418019
72,two_year_recid,0,0.048951,0.794231,0.498258,4.085714,0.652335,1.548148,1.906826,0.825516,0.779269,1.165666


In [100]:
majority_bdf = bias.get_disparity_major_group(xtab, original_df=df)
majority_bdf[['attribute_name', 'attribute_value'] +  bias.list_disparities(majority_bdf)]

,attribute_name,attribute_value,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity
0,sex,Female,0.122195,0.904537,0.941485,1.319947,0.542819,0.867480,0.788406,1.043068,1.059972,1.230259
1,sex,Male,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,age_cat,25 - 45,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,age_cat,Greater than 45,0.109489,0.566139,0.614350,2.686275,0.480769,0.795564,2.055556,0.829596,1.091458,1.163968
4,age_cat,Less than 25,0.532847,1.234574,1.178205,0.220790,3.716216,1.941176,1.372351,0.939890,0.578947,0.142248
...,...,...,...,...,...,...,...,...,...,...,...,...
69,v_score_text,High,1.166667,2.697400,1.223114,0.359447,3.961538,5.967742,0.021184,1.571893,0.000000,0.000000
70,v_score_text,Low,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
71,v_score_text,Medium,1.583333,2.325075,1.149616,0.570458,2.723558,4.397284,0.324206,1.394846,0.316131,0.418019
72,two_year_recid,0,0.048951,0.794231,0.498258,4.085714,0.652335,1.548148,1.906826,0.825516,0.779269,1.165666


## Fairness

In [101]:
from aequitas.fairness import Fairness

f = Fairness()
fdf = f.get_fairness_measures_supported(majority_bdf)

x = f.get_group_value_fairness(majority_bdf)


fdf

get_group_value_fairness: No Parity measure input found on bias_df
get_group_value_fairness: No Parity measure input found on bias_df
get_group_value_fairness: No Parity measure input found on bias_df


['Statistical Parity', 'Impact Parity']